In [1]:
import polars as pl
import plotly.express as px
import toml
from pathlib import Path
import util
import psrc_theme

# to show plotly figures in quarto HTML file
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"
pio.templates.default = "simple_white+psrc_color" # set plotly template

In [2]:
config = toml.load(Path(Path.cwd(), '..\..\..\..\configuration', 'validation_configuration.toml'))
input_config = toml.load(Path(Path.cwd(), '..\..\..\..\configuration', 'input_configuration.toml'))

data = util.ValidationData(config,input_config,['trip'])

In [3]:
mode_cat = {1: "1: walk",
            2: "2: bike",
            3: "3: sov",
            4: "4: hov 2",
            5: "5: hov 3+",
            6: "6: transit",
            8: "8: school bus",
            9: "9:tnc"}
pdpurp_cat = {1: "1: Work",
              2: "2: School",
              3: "3: Escort",
              4: "4: other home-based",
              5: "4: other home-based",
              6: "4: other home-based",
              7: "4: other home-based",
              8: "4: other home-based",
              9: "4: other home-based",
              10: "4: other home-based"}

df_trip = data.trip.with_columns(
    mode_label=pl.col("mode").replace_strict(mode_cat, default=None),
    dpurp_label=pl.col("dpurp").replace_strict(pdpurp_cat, default=None))

In [4]:
def plot_mode_choice(df: pl.DataFrame, title_name: str):
    df_plot = (
        df.group_by(['source', 'dpurp_label', 'mode_label'])
        .agg(
            trexpfac_sum=pl.col('trexpfac').sum(),
            sample_count=pl.col('trexpfac').count()
        )
        .with_columns(
            (pl.col('trexpfac_sum') / pl.col('trexpfac_sum').sum().over(['source', 'dpurp_label'])).alias('percentage')
        )
    )

    fig = px.bar(
        df_plot.sort(by=['dpurp_label', 'source']),
        x="mode_label",
        y="percentage",
        color="source",
        facet_col='dpurp_label',
        facet_col_wrap=2,
        barmode="group",
        hover_data=['sample_count'],
        title=title_name
    )
    fig.update_layout(
        height=600,
        width=900,
        font=dict(size=11),
        xaxis=dict(dtick=1, categoryorder='category ascending')
    )
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    fig.for_each_yaxis(lambda a: a.update(tickformat=".1%"))
    fig.show()

In [5]:
df_trip = df_trip.filter(pl.col('dpurp_label') != 'None')
plot_mode_choice(df_trip,"trip mode choice by purpose")